# Qiita API を使ったキーワード検索

このノートブックでは、Qiita APIを使って記事をキーワード検索し、結果を取得する方法を示します。

参考: [Qiita API v2 ドキュメント](https://qiita.com/api/v2/docs)

## 1. 必要なライブラリのインストールと読み込み

まず、必要なライブラリをインストールします。HTTPリクエストには`requests`を使用します。

In [ ]:
# 必要ライブラリのインストール（初回のみ実行）
# !pip install requests pandas

In [1]:
# 必要なライブラリのインポート
import requests
import pandas as pd
import json
from datetime import datetime

## 2. Qiita API の基本情報

Qiita API v2 のベースURLと各エンドポイントについての基本情報です。
このノートブックでは、認証なしでアクセス可能な検索APIを使用します。

In [2]:
# Qiita API v2のベースURL
BASE_URL = "https://qiita.com/api/v2"

# 検索エンドポイント
SEARCH_ENDPOINT = "/items"

## 3. キーワード検索関数の実装

キーワードをもとにQiita記事を検索する関数を実装します。

In [3]:
def search_qiita_articles(keyword, page=1, per_page=20):
    """
    キーワードを使ってQiitaの記事を検索する関数

    Parameters:
    ----------
    keyword : str
        検索キーワード
    page : int, optional
        取得するページ番号（デフォルト: 1）
    per_page : int, optional
        1ページあたりの記事数（デフォルト: 20、最大: 100）

    Returns:
    -------
    dict
        検索結果とレスポンスヘッダー情報
    """
    # エンドポイントURL
    url = f"{BASE_URL}{SEARCH_ENDPOINT}"

    # クエリパラメータ
    params = {
        'query': keyword,  # 検索キーワード
        'page': page,      # ページ番号
        'per_page': per_page  # 1ページあたりの記事数
    }

    # ヘッダー（User-Agentを設定すると良い）
    headers = {
        'User-Agent': 'QiitaAPITest'
    }

    try:
        # APIリクエスト実行
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()  # エラーチェック

        # レスポンスヘッダーから取得可能な情報
        headers_info = {
            'total_count': int(response.headers.get('Total-Count', 0)), # 検索結果の総記事数
            'rate_limit': response.headers.get('Rate-Limit', None),    # APIレート制限
            'rate_remaining': response.headers.get('Rate-Remaining', None), # 残りのAPI呼び出し回数
            'rate_reset': response.headers.get('Rate-Reset', None)     # レート制限がリセットされる時間
        }

        # レスポンスデータと追加情報を返却
        return {
            'data': response.json(),
            'headers': headers_info
        }

    except requests.exceptions.RequestException as e:
        print(f"APIリクエストエラー: {e}")
        return None

## 4. 検索結果をわかりやすく表示する関数

In [11]:
def display_search_results(results):
    """
    検索結果を整形して表示する関数

    Parameters:
    ----------
    results : dict
        search_qiita_articles関数の戻り値
    """
    if not results:
        print("結果を取得できませんでした")
        return

    # ヘッダー情報の表示
    headers = results['headers']
    print(f"検索結果総数: {headers['total_count']} 件")
    print(f"API レート制限: {headers['rate_limit']} / 残り: {headers['rate_remaining']}")

    if headers['rate_reset']:
        # UNIXタイムスタンプを日時に変換
        reset_time = datetime.fromtimestamp(int(headers['rate_reset']))
        print(f"レート制限リセット時間: {reset_time}\n")

    # 検索結果データ
    articles = results['data']

    if not articles:
        print("検索条件に一致する記事はありませんでした")
        return

    # 必要な項目だけ抽出してDataFrameに変換
    articles_data = []
    for i, article in enumerate(articles, 1):
        articles_data.append({
            '番号': i,
            'タイトル': article['title'],
            'いいね数': article['likes_count'],
            '投稿日': article['created_at'][:10],  # YYYY-MM-DD部分のみ抽出
            '著者': article['user']['id'],
            'URL': article['url'],
            'タグ': ', '.join([tag['name'] for tag in article['tags']])
        })

    # DataFrameに変換して表示
    df = pd.DataFrame(articles_data)

    # 見やすく表示
    pd.set_option('display.max_colwidth', None)  # カラム幅の制限を解除
    display(df)

    return df

## 5. 検索の実行

実際にキーワード検索を実行してみましょう。検索キーワードを変更して試してみてください。

In [ ]:
# 検索キーワードの設定
search_keyword = "Python データ分析"

# 検索実行
results = search_qiita_articles(search_keyword, page=1, per_page=10)

# 結果の表示
df_results = display_search_results(results)

検索結果総数: 15191 件
API レート制限: 60 / 残り: 59
レート制限リセット時間: 2025-06-21 19:19:14



,番号,タイトル,いいね数,投稿日,著者,URL,タグ
0,1,【図解・初心者向け】PandasのDataFrameとSeriesの違いをやさしく解説,0,2025-06-21,coin_collector,https://qiita.com/coin_collector/items/1ed795afdd7310ba42d6,"Python, 初心者, pandas, データ分析, DataFrame"
1,2,独学でデータサイエンスを学ぶ_0/n歩目,0,2025-06-21,t_j_,https://qiita.com/t_j_/items/67ab5d8eafb9c8ef577e,"機械学習, 統計学, データサイエンス, プログラミング初心者"
2,3,[kaggle初心者] Calories予測コンペで機械学習を練習してみた（EDA）,0,2025-06-21,learning_ds,https://qiita.com/learning_ds/items/0d262bdc8dc7e812388e,"Python, 機械学習, Kaggle"
3,4,IT技術ニュース要約 - 2025-06-20,1,2025-06-21,sponge841841,https://qiita.com/sponge841841/items/abf7a841f60c4235a5ae,"daily, manufacturing, Gigazine, tech-news, itmedia"
4,5,化合物をダウンロードして分類・回帰する：chembl-webresource-client × RDKit × RandomForest,2,2025-06-21,maskot1977,https://qiita.com/maskot1977/items/c60b9e864605b8605c9b,"Python, RDKit, randomForest, ChatGPT, chembl_webresource_client"
5,6,歌詞分析プロンプト,0,2025-06-20,digital_amplifier_1,https://qiita.com/digital_amplifier_1/items/44cbe0c0dbc8cb7c4566,プロンプトエンジニアリング
6,7,生成AIはAtCoderBeginnerContestのインフレを加速させたか,11,2025-06-20,sirsoldano,https://qiita.com/sirsoldano/items/ad3d05e4a37759850445,"Python, AtCoder, 競技プログラミング, 生成AI, ChatGPT"
7,8,TMSの理解と構築：輸送管理の最適化への旅 | 第2回：TMSの主要コンポーネント,0,2025-06-20,BNR-John,https://qiita.com/BNR-John/items/8042a945e654533d7e24,"Python, TMS, バイナリテック"
8,9,テクノロジーと社会的不平等 📉: AIが判断する「信用スコア」は差別を助長しないか？,0,2025-06-20,BNR-Gigi,https://qiita.com/BNR-Gigi/items/a7a43880e594a431e6fe,"#バイナリテック, #テックギャップ, #AIと公正, #デジタル格差"
9,10,DatabricksでのMLflow 3.0のクイックスタート,1,2025-06-20,taka_yayoi,https://qiita.com/taka_yayoi/items/ce83575abb55526c52a7,"Databricks, MLflow3.0"


In [19]:
from pprint import pprint
for data in results['data']:
    if data['user']['linkedin_id']:
        print(data['user']['name'])
        print(f"LinkedIn ID: {data['user']['linkedin_id']}")
        # pprint(data['user'])
    if data['organization_url_name']:
        print(data['user']['name'])
        print(f"Organization URL Name: {data['organization_url_name']}")
        # pprint(data['user'])


Ikemen Mas Kot
LinkedIn ID: masaaki-kotera-8699b814/

Organization URL Name: binarytech
Gigi_Codes
Organization URL Name: binarytech
Takaaki Yayoi
LinkedIn ID: takaaki-yayoi
Takaaki Yayoi
Organization URL Name: databricks


## 6. 検索結果の詳細を確認

特定の記事の詳細情報を確認します。

In [6]:
def show_article_details(article_index, results):
    """
    指定された記事の詳細情報を表示する

    Parameters:
    ----------
    article_index : int
        記事のインデックス（0から始まる）
    results : dict
        search_qiita_articles関数の戻り値
    """
    if not results or 'data' not in results:
        print("結果が存在しません")
        return

    articles = results['data']

    if article_index < 0 or article_index >= len(articles):
        print(f"指定されたインデックス {article_index} は範囲外です。0から{len(articles)-1}までの値を指定してください。")
        return

    article = articles[article_index]

    # 詳細情報の表示
    print(f"===== 記事詳細情報 =====\n")
    print(f"タイトル: {article['title']}")
    print(f"著者: {article['user']['id']} (@{article['user']['id']})")
    print(f"投稿日時: {article['created_at']}")
    print(f"更新日時: {article['updated_at']}")
    print(f"いいね数: {article['likes_count']}")
    print(f"ストック数: {article['stocks_count']}")

    # タグの表示
    tags = [f"{tag['name']}" for tag in article['tags']]
    print(f"タグ: {', '.join(tags)}")

    print(f"\nURL: {article['url']}")

    # 本文の冒頭（長すぎる場合は省略）
    body = article['body']
    max_length = 500  # 表示する最大文字数
    if len(body) > max_length:
        body = body[:max_length] + "...(省略)"

    print(f"\n----- 本文の冒頭 -----\n")
    print(body)

In [7]:
# 検索結果の最初の記事（インデックス0）の詳細を表示
show_article_details(0, results)

===== 記事詳細情報 =====

タイトル: 【図解・初心者向け】PandasのDataFrameとSeriesの違いをやさしく解説
著者: coin_collector (@coin_collector)
投稿日時: 2025-06-21T16:39:17+09:00
更新日時: 2025-06-21T16:39:17+09:00
いいね数: 0
ストック数: 0
タグ: Python, 初心者, pandas, データ分析, DataFrame

URL: https://qiita.com/coin_collector/items/1ed795afdd7310ba42d6

----- 本文の冒頭 -----

<h2>初心者向けに「DataFrameとSeriesの違い」を図解で解説！</h2>

<p>
Pythonのデータ分析ライブラリ「Pandas」を使い始めると最初に出会うのが「Series」と「DataFrame」ですが、<strong>この違いが直感的に理解しづらい</strong>という声をよく聞きます。
</p>

<p>
本記事では、<strong>図解・構造比較・コード実行結果</strong>を通じて、初心者でも理解しやすいようにまとめました。
</p>

<h3>DataFrameとSeriesの違い</h3>

<p>以下は構造の違いを示した図です。</p>

<p>
<img src="https://pythondatalab.com/wp-content/uploads/2025/05/pandas_difference_series_dataframe.png" alt="SeriesとDataFrameの構造の違い" width="600" />
</p>

<table>
  <thead>
    <tr><th>構造</th><th>説明</th><...(省略)


## 7. 高度な検索条件の設定

Qiita APIの検索クエリには複雑な条件を指定することができます。
例えば、「Python」に関する記事で、いいねが50以上あるものを検索するなどの複合条件を設定できます。

In [ ]:
# 複合条件を使った検索の例
advanced_query = "Python stocks:>50"  # Pythonに関する記事で、ストック数が50以上

# 検索実行
advanced_results = search_qiita_articles(advanced_query, per_page=5)

# 結果の表示
df_advanced = display_search_results(advanced_results)

## 8. 検索クエリの作成ヘルパー関数

より複雑な検索条件を簡単に構築するためのヘルパー関数を作成します。

In [ ]:
def build_qiita_query(keywords=None, tags=None, user=None, stocks_gt=None, likes_gt=None, created_gt=None):
    """
    Qiita検索用のクエリ文字列を構築するヘルパー関数

    Parameters:
    ----------
    keywords : str or list
        検索キーワード（文字列またはリスト）
    tags : str or list
        検索対象のタグ（文字列またはリスト）
    user : str
        特定のユーザーの記事に限定
    stocks_gt : int
        指定した数以上のストックがある記事
    likes_gt : int
        指定した数以上のいいねがある記事
    created_gt : str
        指定した日付以降に作成された記事（YYYY-MM-DD形式）

    Returns:
    -------
    str
        Qiita API用のクエリ文字列
    """
    query_parts = []

    # キーワード
    if keywords:
        if isinstance(keywords, list):
            query_parts.append(' '.join(keywords))
        else:
            query_parts.append(keywords)

    # タグ
    if tags:
        if isinstance(tags, list):
            for tag in tags:
                query_parts.append(f"tag:{tag}")
        else:
            query_parts.append(f"tag:{tags}")

    # ユーザー
    if user:
        query_parts.append(f"user:{user}")

    # ストック数
    if stocks_gt is not None:
        query_parts.append(f"stocks:>{stocks_gt}")

    # いいね数
    if likes_gt is not None:
        query_parts.append(f"likes:>{likes_gt}")

    # 作成日
    if created_gt:
        query_parts.append(f"created:>{created_gt}")

    # クエリの組み立て
    return ' '.join(query_parts)

In [ ]:
# ヘルパー関数を使った複雑なクエリの作成
complex_query = build_qiita_query(
    keywords="機械学習",
    tags=["Python", "AI"],
    stocks_gt=30,
    created_gt="2023-01-01"
)

print(f"構築されたクエリ文字列: {complex_query}")

# クエリを使用して検索実行
complex_results = search_qiita_articles(complex_query, per_page=5)

# 結果の表示
df_complex = display_search_results(complex_results)

## 9. まとめ

このノートブックでは、Qiita APIを使用して記事を検索し、結果を取得・表示する方法を紹介しました。

主な機能:
- キーワードによる記事検索
- 検索結果の整形表示
- 詳細記事情報の取得
- 複雑な検索条件の構築

注意点:
- 認証なしの場合、APIの呼び出し回数に制限があります (現在は60回/時間)
- より高度な機能やアクセス制限の緩和には、Qiitaへのユーザー登録とアクセストークンの取得が必要です

さらに詳しい情報は [Qiita API v2 ドキュメント](https://qiita.com/api/v2/docs) をご参照ください。